In [1]:
# elasticsearch 서버 접속 모듈 설치
# pip install elasticsearch

from elasticsearch import Elasticsearch
from elasticsearch import helpers

# elastic 서버 접속 
#es = Elasticsearch("https://192.168.0.91:9200/", verify_certs=False)
es = Elasticsearch("http://

/MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'name': 'node-1',
 'cluster_name': 'mpower',
 'cluster_uuid': 'Nnk0icM8QeWRIRqqTYjiTA',
 'version': {'number': '7.17.3',
  'build_flavor': 'default',
  'build_type': 'tar',
  'build_hash': '5ad023604c8d7416c9eb6c0eadb62b14e766caff',
  'build_date': '2022-04-19T08:11:19.070913226Z',
  'build_snapshot': False,
  'lucene_version': '8.11.1',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [2]:
# 삭제, 업데이트, 추가, 쿼리 등의 elastricsearch 함수들 정의
# => 참고 : https://jvvp.tistory.com/1152

###########################################################
# 인덱스 생성/삭제
###########################################################
## 인덱스 생성
def create_index(index, mapping=None):
    if not es.indices.exists(index=index):
        return es.indices.create(index=index ,body=mapping)

## 인덱스 자체 삭제
def delete_index(index):
    if es.indices.exists(index=index):
        return es.indices.delete(index=index)
    
###########################################################
# 인데스에 데이터 추가 
###########################################################
def insert(index, doc_type, body):
    return es.index(index=index, doc_type=doc_type, body=body)


###########################################################
# 인덱스내 데이터 조회 => query 이용
###########################################################
def search(index, data=None):
    if data is None: #모든 데이터 조회
        data = {"match_all":{}}
    else:
        data = {"match": data}
        
    body = {"query": data}
    res = es.search(index=index, body=body)
    return res
###########################################################

############################################################
## 인덱스 내의 데이터 삭제 => query 이용
############################################################
def delete(index, data):
    if data is None:  # data가 없으면 모두 삭제
        data = {"match_all":{}}
    else:
        data = {"match": data}
        
    body = {"query": data}
    return es.delete_by_query(index=index, body=body)

############################################################
## 인덱스 내의 데이터 삭제 => id 이용
############################################################
def delete_by_id(index, id):
    return es.delete(index=index, id=id)

############################################################
## 인덱스 내의 데이터 업데이트=>_id 에 데이터 업데이트
############################################################
def update(index, id, doc, doc_type):
    
    body = {
        'doc': doc
    }
    
    res=es.update(index=index, id=id, body=body, doc_type=doc_type)
    return res
############################################################


In [ ]:
# 인덱스 살제
index_name = 'goods'
response=delete_index(index=index_name)
print(f'rs:{response}')

In [ ]:
# 인덱스 생성
index_name = 'goods'

# 매핑 정의
# => 매핑을 정의하면, 데이터타입을 지정해 줄수 있다.
float_type=768
mapping =
{
  "settings": {
    "number_of_shards": 2,
    "number_of_replicas": 1
  },
   "mappings": {
    "dynamic": "true",
    "_source": {
      "enabled": "true"
    },
    "properties": {
      "title": {
        "type": "text"
      },
	  "paragraph": {
        "type": "text"
      },
      "title_vector": {
        "type": "dense_vector",
        "dims": 768
      },
	  "paragraph_vector": {
        "type": "dense_vector",
        "dims": float_type
      }
    }
  }
}


response = create_index(index=index_name, mapping=mapping)
print(f'rs:{response}')


In [ ]:
# 인덱스에 데이터 추가
index_name = 'goods'

doc1 = {'title':'제주도', 'paragraph':'제주도는 대한민국이 남쪽에 있는 섬이다.'}
doc2 = {'title':'한라산', 'paragraph':'한라산은 제주도에 있는 휴화산이다.'}
doc3 = {'title':'서귀포', 'paragraph':'대한민국 제주도 최남단에 있는 도시이다.'}
doc4 = {'title':'제주시', 'paragraph':'제주도의 행정/경제 중심 도시이다.'}
doc5 = {'title':'성산읍', 'paragraph':'제주도 동쪽에 있는 읍으로, 성산일출봉과 제2공항 후보지로 유명하다.'}

res=insert(index=index_name, doc_type="_doc", body=doc1)
print(f'rs:{res}')

In [ ]:
# 검색
index_name = 'goods'

# 모든 데이터 검색
res=search(index_name, data=None)
print(f'rs:{res}')
print('\n')

# 쿼리 검색
data = {'title': '제주도'}
res=search(index_name, data=data)
print(f'rs:{res}')


In [ ]:
# 인덱스에 데이터 삭제

index_name = 'goods'

'''
# 쿼리로 삭제
data = {'title': '서귀포'}
res=delete(index=index_name, data=data)
print(f'res:{res}')
'''


# 인덱스로 삭제
# 쿼리 검색
data = {'title': '제주도'}
res=search(index_name, data=data)
for hits in res['hits']['hits']:
    id=hits['_id']
    #print(f'id:{id}')
    break
    
print(f'id:{id}')
res=delete_by_id(index=index_name, id=id)
print(f'res:{res}')


In [ ]:
# 인덱스에 데이터 업데이트
# =>업데이트할 _id, _type을 알아야함

index_name = 'goods'

# 쿼리 검색
data = {'title': '제주도'}
res=search(index_name, data=data)
print(f'rs:{res}')
print('\n')

id_list = []
type_list = []
# 업데이트할 id와 type을 구함
for hits in res['hits']['hits']:
    id=hits['_id']
    type=hits['_type']
    #print(f'id:{id}, type:{type}')
    
    id_list.append(id)
    type_list.append(type)
    
# 업데이트
for id, type in zip(id_list, type_list):
    
    print(f'id:{id}, type:{type}')
    doc = {'paragraph': '제주도는 대한민국에 가장 남쪽에 있는 섬으로, 인구는 약 70만명이며, 화산섬으로 관광자원이 많은 천혜의 관광지 이다.'}
    
    res=update(index=index_name, doc=doc, id=id, doc_type=type)
    print(f'res:{res}')
    print('\n')
    
# 인덱스 refresh 함
# elasticsearch의 자동 새로고침의 시간은 1초 정도 소요
# 따라서 코드에 아래 명령어를 입력하지 않았을 경우 검색을 하지 못할 가능성도 존재
es.indices.refresh(index=index_name)

# 쿼리 검색
data = {'title': '제주도'}
res=search(index_name, data=data)
print(f'rs:{res}')
print('\n')